!pip install -U ydata-profiling -t /home/aistudio/external-libraries

!pip install --upgrade jupyter ipywidgets -t /home/aistudio/external-libraries

!pip install openpyxl -t /home/aistudio/external-libraries

In [1]:
import pandas as pd
import sys 
sys.path.append('/home/aistudio/external-libraries')

df1 = pd.read_excel('/home/aistudio/data/data237885/附件1.xlsx')

In [2]:
df2 = pd.read_excel('/home/aistudio/data/data237885/附件2.xlsx')

In [3]:
df3 = pd.read_excel('/home/aistudio/data/data237885/附件3.xlsx')
df4 = pd.read_excel('/home/aistudio/data/data237885/附件4.xlsx')

In [4]:
import calendar
import numpy as np

# 使用单品编码进行合并，并且只保留附件2中已有的单品编码
merged_df = pd.merge(df2, df1[['单品编码', '单品名称', '分类名称']], on='单品编码', how='left')

merged_df = pd.merge(merged_df, df3[['日期', '单品编码', '批发价格(元/千克)']], left_on=['销售日期', '单品编码'], right_on=['日期', '单品编码'], how='left')

# 添加一个序号列，从1开始
merged_df['序号'] = np.arange(1, len(merged_df) + 1)

# 如果你想让'序号'列成为第一列，你可以这样重新排列列的顺序：
cols = ['序号'] + [col for col in merged_df.columns if col != '序号']
merged_df = merged_df[cols]

# 转为日期格式
merged_df['销售日期'] = pd.to_datetime(merged_df['销售日期'])

# 添加“星期”列（1-7的数字表示）
merged_df['星期'] = pd.to_datetime(merged_df['销售日期']).dt.dayofweek + 1  # +1是为了让星期一开始为1

# 添加“年中进度”列
merged_df['年中进度'] = merged_df['销售日期'].apply(
    lambda x: x.timetuple().tm_yday / (366 if calendar.isleap(x.year) else 365)
)

# 新增一个表示所在周的列
merged_df['周'] = merged_df['销售日期'].dt.to_period('W')

# 新增一个表示所在月份的列
merged_df['月份'] = merged_df['销售日期'].dt.to_period('M')

# 创建一个新的列"季度"
merged_df['季度'] = merged_df['销售日期'].dt.to_period('Q')

In [5]:
def time_to_day_fraction(time_str):
    from datetime import datetime
    try:
        time_obj = datetime.strptime(time_str, '%H:%M:%S.%f').time()
    except ValueError:
        time_obj = datetime.strptime(time_str, '%H:%M:%S').time()

    total_seconds = time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second + time_obj.microsecond * 1e-6
    return total_seconds / 86400  # 86400 = 24*60*60

merged_df['扫码销售时间'] = merged_df['扫码销售时间'].apply(time_to_day_fraction)

In [6]:
# 将销售日期转换为字符串格式，以保持原来的格式（只包含日期部分）
merged_df['销售日期'] = merged_df['销售日期'].dt.strftime('%Y-%m-%d')

# 添加“交易金额”列
merged_df['交易金额'] = merged_df['销量(千克)'] * merged_df['销售单价(元/千克)']

# 添加新的"成本"列
merged_df['成本'] = merged_df['销量(千克)'] * merged_df['批发价格(元/千克)']

# 重新计算“利润”列
merged_df['利润'] = merged_df['交易金额'] - merged_df['成本']

# 添加"成本加成定价"列
merged_df['利润率'] = merged_df['利润'] / merged_df['成本']

# 将“销售类型”列中的“销售”赋值为1，“退货”赋值为-1
merged_df['销售类型'] = merged_df['销售类型'].replace({'销售': 1, '退货': -1})

# 将“是否打折销售”列中的“是”赋值为1，“否”赋值为0
merged_df['是否打折销售'] = merged_df['是否打折销售'].replace({'是': 1, '否': 0})

In [7]:
merged_df.drop('日期', axis=1, inplace=True)
merged_df.head()

,序号,销售日期,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售,单品名称,分类名称,批发价格(元/千克),星期,年中进度,周,月份,季度,交易金额,成本,利润,利润率
0,1,2020-07-01,0.385508,102900005117056,0.396,7.6,1,0,泡泡椒(精品),辣椒类,4.32,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,3.0096,1.71072,1.29888,0.759259
1,2,2020-07-01,0.387121,102900005115960,0.849,3.2,1,0,大白菜,花叶类,2.10,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,2.7168,1.78290,0.93390,0.523810
2,3,2020-07-01,0.387198,102900005117056,0.409,7.6,1,0,泡泡椒(精品),辣椒类,4.32,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,3.1084,1.76688,1.34152,0.759259
3,4,2020-07-01,0.388720,102900005115823,0.421,10.0,1,0,上海青,花叶类,7.03,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,4.2100,2.95963,1.25037,0.422475
4,5,2020-07-01,0.389163,102900005115908,0.539,8.0,1,0,菜心,花叶类,4.60,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,4.3120,2.47940,1.83260,0.739130


In [8]:
def QutlierReplacement(df, col, percentage=0.02):
    """
    传入DataFrame对象和需要处理异常值的列名。
    如果异常值的数据量小于数据量*百分比，就替换数据。
    """
    processed_records = {}  
    
    df = df.reset_index(drop=True)
    df_col = df[col]  # Get column
    df_col_value = df[col].values
    
    Q1 = df_col.quantile(q=0.25)
    Q3 = df_col.quantile(q=0.75)
    
    up_whisker = Q3 + 5 * (Q3 - Q1)
    low_whisker = Q1 - 5 * (Q3 - Q1)  
    
    right = set(np.where(df_col_value > up_whisker)[0])  
    left = set(np.where(df_col_value < low_whisker)[0])  
    
    choose = list(right | left)  
    
    if len(choose) < len(df) * percentage:
        for idx in choose:
            original_value = df.at[idx, col]
            item_name = df.at[idx, '单品名称']
            
            mean_value = df[df['单品名称'] == item_name][col].mean()
            
            df.at[idx, col] = mean_value  
            processed_records[df.at[idx, '序号']] = (original_value, df.at[idx, col])
    
    return df, processed_records 

# 使用该函数处理merged_df中的“销量(千克)”列
merged_df, processed_records = QutlierReplacement(merged_df, '销量(千克)')

count = 0
# 输出被处理的记录
for seq_no, (original_value, new_value) in processed_records.items():
    print(f"序号: {seq_no}, 处理前销量(千克): {original_value}, 处理后销量(千克): {new_value}")
    count += 1
    
print(count)

序号: 606217, 处理前销量(千克): 5.0, 处理后销量(千克): 4.2408906882591095
序号: 638988, 处理前销量(千克): 5.0, 处理后销量(千克): 4.2393540297333185
序号: 628749, 处理前销量(千克): 5.0, 处理后销量(千克): 4.237814260562738
序号: 618515, 处理前销量(千克): 5.0, 处理后销量(千克): 4.236271374450517
序号: 630803, 处理前销量(千克): 5.0, 处理后销量(千克): 4.234725365087057
序号: 606231, 处理前销量(千克): 5.0, 处理后销量(千克): 4.233176226149985
序号: 618519, 处理前销量(千克): 5.0, 处理后销量(千克): 4.231623951304134
序号: 612380, 处理前销量(千克): 5.0, 处理后销量(千克): 4.2300685342015125
序号: 616476, 处理前销量(千克): 5.0, 处理后销量(千克): 4.228509968481272
序号: 614430, 处理前销量(千克): 10.0, 处理后销量(千克): 4.226948247769696
序号: 247842, 处理前销量(千克): -2.273, 处理后销量(千克): 1.2641466596389512
序号: 581667, 处理前销量(千克): 4.141, 处理后销量(千克): 0.447347337204992
序号: 620583, 处理前销量(千克): 5.0, 处理后销量(千克): 4.215261908190282
序号: 157740, 处理前销量(千克): 7.551, 处理后销量(千克): 1.2643797039570195
序号: 649260, 处理前销量(千克): 5.0, 处理后销量(千克): 4.213673369542894
序号: 157742, 处理前销量(千克): 9.694, 处理后销量(千克): 1.2639655110267227
序号: 694316, 处理前销量(千克): 3.03, 处理后销量(千克): 1.0119750312109863
序号: 655414, 处

In [9]:
merged_df.head()

,序号,销售日期,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售,单品名称,分类名称,批发价格(元/千克),星期,年中进度,周,月份,季度,交易金额,成本,利润,利润率
0,1,2020-07-01,0.385508,102900005117056,0.396,7.6,1,0,泡泡椒(精品),辣椒类,4.32,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,3.0096,1.71072,1.29888,0.759259
1,2,2020-07-01,0.387121,102900005115960,0.849,3.2,1,0,大白菜,花叶类,2.10,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,2.7168,1.78290,0.93390,0.523810
2,3,2020-07-01,0.387198,102900005117056,0.409,7.6,1,0,泡泡椒(精品),辣椒类,4.32,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,3.1084,1.76688,1.34152,0.759259
3,4,2020-07-01,0.388720,102900005115823,0.421,10.0,1,0,上海青,花叶类,7.03,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,4.2100,2.95963,1.25037,0.422475
4,5,2020-07-01,0.389163,102900005115908,0.539,8.0,1,0,菜心,花叶类,4.60,3,0.5,2020-06-29/2020-07-05,2020-07,2020Q3,4.3120,2.47940,1.83260,0.739130


In [10]:
merged_df.describe()

,序号,扫码销售时间,单品编码,销量(千克),销售单价(元/千克),销售类型,是否打折销售,批发价格(元/千克),星期,年中进度,交易金额,成本,利润,利润率
count,878503.000000,878503.000000,8.785030e+05,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000,878503.000000
mean,439252.000000,0.621333,1.030313e+14,0.533873,8.917144,0.998950,0.053917,5.663150,4.268100,0.505362,3.835805,2.420156,1.415649,0.766252
std,253602.116104,0.152056,7.174368e+11,0.335764,6.311265,0.045803,0.225853,4.229638,2.041563,0.287668,2.813984,1.904971,1.249791,7.960573
min,1.000000,0.343202,1.029000e+14,-1.768000,0.100000,-1.000000,0.000000,0.010000,1.000000,0.002740,-100.000000,-21.368680,-161.900000,-0.967480
25%,219626.500000,0.467446,1.029000e+14,0.290000,4.900000,1.000000,0.000000,3.000000,2.000000,0.246575,2.210000,1.330290,0.733405,0.444043
50%,439252.000000,0.642595,1.029000e+14,0.435000,7.900000,1.000000,0.000000,4.690000,4.000000,0.538251,3.332000,2.066100,1.195600,0.595793
75%,658877.500000,0.752034,1.029000e+14,0.741000,10.800000,1.000000,0.000000,7.010000,6.000000,0.745902,4.791600,3.021030,1.859550,0.773399
max,878503.000000,0.966528,1.069740e+14,4.878049,119.900000,1.000000,1.000000,141.000000,7.000000,1.000000,944.000000,523.200000,420.800000,1599.000000


In [11]:
# 将合并后的DataFrame保存为新的Excel文件
merged_df.to_excel("/home/aistudio/work/附件2(处理后).xlsx", index=False)

In [12]:
# 检查“销售类型”是“销售”，但“销量(千克)”不是正数的情况
mask1 = (merged_df['销售类型'] == '销售') & (merged_df['销量(千克)'] <= 0)
# 检查“销售类型”是“退货”，但“销量(千克)”不是负数的情况
mask2 = (merged_df['销售类型'] == '退货') & (merged_df['销量(千克)'] >= 0)

# 将不符合条件的行选出来
invalid_rows1 = merged_df[mask1]
invalid_rows2 = merged_df[mask2]

if len(invalid_rows1) > 0:
    print("以下行的‘销售类型’是‘销售’，但‘销量(千克)’不是正数：")
    print(invalid_rows1)
if len(invalid_rows2) > 0:
    print("以下行的‘销售类型’是‘退货’，但‘销量(千克)’不是负数：")
    print(invalid_rows2)

if len(invalid_rows1) == 0 and len(invalid_rows2) == 0:
    print("所有数据都满足条件。")

所有数据都满足条件。


In [13]:
# 转换'销售日期'列为日期格式，并提取月份信息
merged_df['销售日期'] = pd.to_datetime(merged_df['销售日期'])
merged_df['月份'] = merged_df['销售日期'].dt.to_period('M')  # 将日期转换为月份

# 按照月份和分类名称分组，然后计算每组的“销量(千克)”和“利润”的总和和“成本加成定价”的均值
grouped_df = merged_df.groupby(['月份', '分类名称']).agg({'销量(千克)': 'sum', '利润': 'sum','利润率':'mean'}).reset_index()

# 转换为宽格式
pivot_df = grouped_df.pivot(index='月份', columns='分类名称', values=['销量(千克)', '利润','利润率'])

# 重命名列名以便于理解
pivot_df.columns = [f"{col[1]}{col[0]}" for col in pivot_df.columns]

# 将月份列的数据类型转换为字符串，并格式化为"YYYY-MM"格式
pivot_df.index = pivot_df.index.strftime('%Y-%m')

# 保存为新的Excel文件
pivot_df.to_excel("/home/aistudio/work/每月各品类销量及利润分析.xlsx")

In [14]:
pivot_df.head()

,水生根茎类销量(千克),花叶类销量(千克),花菜类销量(千克),茄类销量(千克),辣椒类销量(千克),食用菌销量(千克),水生根茎类利润,花叶类利润,花菜类利润,茄类利润,辣椒类利润,食用菌利润,水生根茎类利润率,花叶类利润率,花菜类利润率,茄类利润率,辣椒类利润率,食用菌利润率
月份,,,,,,,,,,,,,,,,,,
2020-07,332.056000,6583.075665,1522.555000,1359.174475,2198.049000,1617.836000,1906.95755,19385.42505,6429.03766,3872.97233,9338.89707,8108.16282,0.778408,0.676551,0.669053,0.739938,0.726496,0.721115
2020-08,861.872190,7261.517000,1748.658000,1139.393000,2822.344000,1636.524000,3027.05693,22006.66348,6463.90490,3298.22782,8456.13838,8058.56206,0.573673,0.694277,0.582266,0.733294,0.595257,0.730852
2020-09,798.931000,5636.198942,1322.988825,671.692000,1976.160634,1642.881000,1975.99966,17584.48696,5310.45573,1792.22049,6564.35024,7985.83713,0.470644,0.783640,0.600075,0.547475,0.809770,0.730827
2020-10,1338.835398,6326.418284,1644.963000,771.967000,1908.944000,2997.612986,2863.13098,17956.17865,5892.33508,1628.85046,6979.71456,11497.46273,0.380920,0.685543,0.615605,0.500914,0.546906,0.750557
2020-11,958.212822,6294.975565,1943.391000,432.389000,1837.705000,3281.028000,1968.12191,9991.13511,3446.86358,906.69904,5422.49353,8500.04485,0.357084,0.705345,0.492527,0.487573,0.467573,0.623124


In [15]:
# 按照销售日期、星期和分类名称分组，然后计算每组的“销量(千克)”和“利润”的总和
grouped_df = merged_df.groupby(['销售日期', '星期', '分类名称']).agg({'销量(千克)': 'sum', '利润': 'sum','利润率':'mean'}).reset_index()

# 转换为宽格式
pivot_df = grouped_df.pivot(index=['销售日期', '星期'], columns='分类名称', values=['销量(千克)', '利润','利润率'])

# 重命名列名以便于理解
pivot_df.columns = [f"{col[1]}_{col[0]}" for col in pivot_df.columns]

# 由于索引现在是多层次的（销售日期和星期），我们只将销售日期列的数据类型转换为字符串，并格式化为"YYYY-MM-DD"格式
pivot_df.index = pd.MultiIndex.from_tuples([(idx[0].strftime('%Y-%m-%d'), idx[1]) for idx in pivot_df.index])

# 保存为新的Excel文件
pivot_df.to_excel("/home/aistudio/work/每日各品类销量及利润分析.xlsx")

In [16]:
pivot_df.head()

,,水生根茎类_销量(千克),花叶类_销量(千克),花菜类_销量(千克),茄类_销量(千克),辣椒类_销量(千克),食用菌_销量(千克),水生根茎类_利润,花叶类_利润,花菜类_利润,茄类_利润,辣椒类_利润,食用菌_利润,水生根茎类_利润率,花叶类_利润率,花菜类_利润率,茄类_利润率,辣椒类_利润率,食用菌_利润率
2020-07-01,3,4.850,205.402,46.640,35.374,76.715,35.365,25.49812,512.10105,200.71800,32.08965,351.93465,138.66059,0.714238,0.526693,0.511141,0.632999,0.876177,0.597930
2020-07-02,4,4.600,198.362,43.943,32.199,66.064,48.510,20.48956,583.96795,201.69822,92.46514,292.84857,251.18003,0.703021,0.711268,0.591160,1.341366,0.806968,0.680904
2020-07-03,5,9.572,190.779,42.076,35.896,64.253,42.442,22.99408,561.73894,182.90884,53.94768,268.84345,234.01637,0.461676,0.675462,0.594906,0.755759,0.725987,0.743286
2020-07-04,6,5.439,236.587,55.662,57.067,81.282,47.262,42.52497,704.10491,244.78158,149.64214,361.91539,232.02942,0.685346,0.624207,0.553579,0.860294,0.837712,0.670419
2020-07-05,7,4.019,223.899,55.474,61.816,98.496,73.213,18.40702,616.37082,232.15808,162.74564,421.20867,301.72841,0.845018,0.651440,0.511039,0.716825,0.786806,0.714895


In [17]:
# 创建一个日期范围从2020-07-01到2023-06-30
all_dates = pd.date_range(start='2020-07-01', end='2023-06-30')

# 查找在'销售日期'列中存在的日期
existing_dates = pd.to_datetime(merged_df['销售日期'].unique())

# 使用Pandas的Index差集功能来找到缺失的日期
missing_dates = all_dates.difference(existing_dates)

# 打印或输出缺失的日期
print("缺失的日期如下：")
print(missing_dates)

缺失的日期如下：
DatetimeIndex(['2021-02-11', '2021-02-12', '2022-01-31', '2022-11-02',
               '2022-11-04', '2022-11-30', '2022-12-01', '2022-12-02',
               '2022-12-03', '2023-01-21'],
              dtype='datetime64[ns]', freq=None)


## 一键快速预览数据

import numpy as np
from ydata_profiling import ProfileReport

#创建数据分析报告
profile = ProfileReport(merged_df, title='Your Data Analysis Report', explorative=True)

#保存报告为HTML文件
profile.to_file("/home/aistudio/data/data237885/merged_ydata_report.html")

#显示报告（适用于Jupyter Notebook）
profile.to_notebook_iframe()

In [18]:
from datetime import datetime

# 定义数据中最早和最晚的两个日期字符串
date_str1 = '2020-07-01 00:00:00'
date_str2 = '2023-06-30 00:00:00'

# 将日期字符串转换为 datetime 对象
date1 = datetime.strptime(date_str1, '%Y-%m-%d %H:%M:%S')
date2 = datetime.strptime(date_str2, '%Y-%m-%d %H:%M:%S')

# 计算两个日期之间的天数差
delta = date2 - date1

# 输出天数
print(f"间隔天数: {delta.days}（天）")

间隔天数: 1094（天）


In [19]:
temp_merged_df = merged_df.copy()

print(temp_merged_df.shape)

(878503, 20)


In [20]:
temp_merged_df = merged_df.copy()

# 按照“销售日期”和“单品名称”进行分组，计算每天各单品的总“销量(千克)”
grouped_df = temp_merged_df.groupby(['销售日期', '单品名称'])['销量(千克)'].sum().reset_index()

# 使用 df1 中的全部单品名称，确保所有单品都在结果中出现
# 创建一个唯一的销售日期列表和单品名称列表
unique_dates = temp_merged_df['销售日期'].unique()
unique_products = df1['单品名称'].unique()
# 创建一个全排列的 DataFrame，其中包含所有可能的“销售日期”和“单品名称”组合
import itertools
all_combinations = pd.DataFrame(list(itertools.product(unique_dates, unique_products)), columns=['销售日期', '单品名称'])

# 将全排列的 DataFrame 与 grouped_df 合并，以获取每个单品每天的销量（如果没有记录，则为 0）
final_df = pd.merge(all_combinations, grouped_df, on=['销售日期', '单品名称'], how='left').fillna(0)
# 添加“分类名称”列
final_df = pd.merge(final_df, df1[['单品名称', '分类名称']], on='单品名称', how='left')

# 在转换日期格式之前添加“星期”列
final_df['星期'] = final_df['销售日期'].dt.dayofweek + 1 

final_df['销售日期'] = final_df['销售日期'].dt.date

output_path = '/home/aistudio/work/每日各单品总销量情况.xlsx'
final_df.to_excel(output_path, index=False)

In [21]:
# 使用 pivot_table 方法进行转置
pivot_df = final_df.pivot_table(index=['销售日期', '星期'], columns='单品名称', values='销量(千克)', fill_value=0).reset_index()

# 如果您需要，可以再次将 '销售日期' 列转换为仅日期格式（这取决于数据当前的格式）
pivot_df['销售日期'] = pd.to_datetime(pivot_df['销售日期']).dt.date

# 保存为新的 Excel 文件
output_path_transposed = '/home/aistudio/work/每日各单品总销量情况（转置）.xlsx'
pivot_df.to_excel(output_path_transposed, index=False)

## 处理附件3

In [22]:
# 使用Pandas的dt属性来获取日期对应的星期（星期一为0，星期日为6）
df3['星期'] = df3['日期'].dt.dayofweek + 1  # 将星期一作为1，星期日作为7

# 使用“单品编码”列来合并df1和df3，以便在df3中添加“分类名称”列
df3_merged = pd.merge(df3, df1[['单品编码', '分类名称']], on='单品编码', how='left')

# 转换日期列为只包含日期的字符串格式
df3_merged['日期'] = df3_merged['日期'].dt.strftime('%Y-%m-%d')

df3_merged.to_excel('/home/aistudio/work/附件3(处理后).xlsx', index=False)

In [23]:
df3_merged.head()

,日期,单品编码,批发价格(元/千克),星期,分类名称
0,2020-07-01,102900005115762,3.88,3,花叶类
1,2020-07-01,102900005115779,6.72,3,花叶类
2,2020-07-01,102900005115786,3.19,3,花叶类
3,2020-07-01,102900005115793,9.24,3,花叶类
4,2020-07-01,102900005115823,7.03,3,花叶类
